# Emissions

Data sourced from [Jacques de Chalendar's work](https://gridemissions.jdechalendar.su.domains/#/code).

#### Data sources
- Hourly average emissions by balancing authority: https://gridemissions.jdechalendar.su.domains/#/code
- State-to-BA mappings: RMI
  - https://docs.google.com/spreadsheets/d/1seafDzJwZfBkdBslKpWwN2hsOmUgrV-N2Vn1L4XfMK4/edit#gid=1175680063
  - http://rmi.org/wp-content/uploads/2020/06/Emissions-Results-1.pdf
- ZIP-to-BA mappings: WattTime
  - https://www.watttime.org/explorer/#4/39.03/-106.38
  

#### Steps:
1. Download bulk data from https://gridemissions.jdechalendar.su.domains/#/code
  - Hourly emissions embodied in the electric sector (July 2018 to present) - https://gridemissions.s3.us-east-2.amazonaws.com/EBA_co2.csv.gz
  - Reconciled hourly electricity data (July 2018 to present) -- https://gridemissions.s3.us-east-2.amazonaws.com/EBA_elec.csv.gz
  - Refer to data naming conventions in https://github.com/jdechalendar/gridemissions/
2. Clean and select what's relevant
  - CO2: "D": "CO2_%s_D" (demand / consumption) --> tons
  - Electricty: "EBA.%s-ALL.D.H" (demand / consumption) --> GWh


In [1]:
import csv
import json
import pprint
from datetime import datetime, timedelta

In [2]:
def prep_grid_emissions_data():
    """
    Utility function to clean raw emissions and electricity consumption data.

    Outputs a single .csv of lbs co2e per kwh of electricity consumed for each balancing authority.
    Input data downloaded from https://gridemissions.jdechalendar.su.domains/#/code.
    """

    # load raw files
    print("Loading raw data...")
    with open("data/emissions/raw/EBA_elec.csv") as file:
        reader = csv.DictReader(file)
        raw_elec_data = [x for x in reader]

    with open("data/emissions/raw/EBA_co2.csv") as file:
        reader = csv.DictReader(file)
        raw_co2_data = [x for x in reader]
        
    print("Cleaning raw data...")

    # first column title from raw data download is blank, so give it a name
    # 2018-07-01 00:00:00+00:00
    for x in raw_elec_data:
        x['ts'] = x.pop('')
    for x in raw_co2_data:
        x['ts'] = x.pop('')
        
    # select a subset of years (raw download have July 2018 to present)
    years = ['2022', '2023']
    raw_elec_data = [x for x in raw_elec_data if x['ts'][:4] in years]
    raw_co2_data = [x for x in raw_co2_data if x['ts'][:4] in years]
    
    # clean column titles to just the balancing authority abbreviation and
    # extract demand (consumption) data
    # - electricity format: "EBA.%s-ALL.D.H"
    # - co2 format:         "CO2_%s_D"
    elec = []
    for row in raw_elec_data:
        trim_row = {'ts': row['ts']}
        for key in row:
            if key != 'ts':
                if key[-8:] == '-ALL.D.H':
                    bal_auth = key[:-8][4:]
                    trim_row[bal_auth] = row[key]
        elec.append(trim_row)

    co2 = []
    for row in raw_co2_data:
        trim_row = {'ts': row['ts']}
        for key in row:
            if key != 'ts':
                if key[-2:] == "_D":
                    bal_auth = key[:-2][4:]
                    trim_row[bal_auth] = row[key]
        co2.append(trim_row)
        
                
    # merge data and calculate lbs per kwh
    print("Merging raw data...")
    merged_data = []
    for erow in elec:
        ts = erow['ts']
        mrow = {'ts': ts}
        
        # get corresponding row in co2
        crow = [x for x in co2 if x['ts'] == ts][0]
        
        # for each balancing authority
        # e_usage == GWh --> * 1000000
        # c_usage == tons --> * 2000
        for bal_auth, e_gwh in erow.items():
            if bal_auth != 'ts':
                e_kwh = float(e_gwh) * 1000000 if e_gwh else 'na'
                c_lbs = float(crow[bal_auth]) * 2000 if crow[bal_auth] else 'na'

                lbs_per_kwh = c_lbs / e_kwh if c_lbs != 'na' and e_kwh != 'na' and e_kwh > 0 else "na"

                mrow[bal_auth] = lbs_per_kwh        
        merged_data.append(mrow)
    
    # save output
    print("Saving output...")
    
    # for all ba's
    keys = list(merged_data[0].keys())
    with open('data/emissions/clean/avg_hourly_emissions.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(merged_data)
        
    with open('data/emissions/clean/avg_hourly_emissions.json', 'w') as output_file:
        json.dump(merged_data, output_file, indent=4)
    
    # by balance authority, to make loading faster
    balance_authorities = list(merged_data[0].keys())
    balance_authorities = [x for x in balance_authorities if x != 'ts']
    
    for balauth in balance_authorities:
        data = []
        for row in merged_data:
            data.append({
                'ts': row['ts'],
                'avg': row[balauth],
            })
        keys = list(data[0].keys())
        with open(f'data/emissions/clean/avg_hourly_emissions/{balauth}.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data)
    
    print("Done.")
    return

prep_grid_emissions_data()

Loading raw data...
Cleaning raw data...
Merging raw data...
Saving output...
Done.
